In [ ]:
# Supplier Reliability: % of Late Shipments

   # Query to calculate % of late shipments per supplier
query = """
SELECT s.company_name AS supplier,
       ROUND(100.0 * SUM(CASE WHEN o.shipped_date > o.required_date THEN 1 ELSE 0 END)::numeric / COUNT(*), 2) AS pct_late
FROM suppliers s
JOIN products p ON s.supplier_id = p.supplier_id
JOIN order_details od ON p.product_id = od.product_id
JOIN orders o ON od.order_id = o.order_id
WHERE o.shipped_date IS NOT NULL AND o.required_date IS NOT NULL
GROUP BY s.company_name
ORDER BY pct_late DESC;
"""

df = pd.read_sql(query, engine)
df = df.sort_values('pct_late', ascending=True)

# Plot horizontal bar chart
fig = px.bar(df, x="pct_late", y="supplier", orientation="h",
             labels={"pct_late": "% Late Shipments", "supplier": "Supplier"},
             title="Supplier Reliability: % of Late Shipments")
fig.update_layout(xaxis_tickformat=".1f", xaxis_range=[0, max(df["pct_late"].max(), 10)])
fig.show()

In [ ]:
# Lead Time per Supplier

  # Query to get Lead Time
query = """
SELECT 
    s.company_name AS supplier_name,
    o.order_id,
    o.order_date,
    o.shipped_date,
    (o.shipped_date - o.order_date)::int AS lead_time_days
FROM orders o
JOIN order_details od ON o.order_id = od.order_id
JOIN products p ON od.product_id = p.product_id
JOIN suppliers s ON p.supplier_id = s.supplier_id
WHERE o.shipped_date IS NOT NULL AND o.order_date IS NOT NULL
"""
df = pd.read_sql_query(query, engine)
df = df[df['lead_time_days'] >= 0]  # Remove negative or invalid lead times

# Aggregate average lead time per supplier
avg_lead_time = df.groupby('supplier_name', as_index=False)['lead_time_days'].mean()
avg_lead_time = avg_lead_time.sort_values('lead_time_days', ascending=False)

# Plot using Plotly
fig = px.bar(
    avg_lead_time,x='supplier_name', y='lead_time_days',
    title='Average Lead Time by Supplier (Days)',
    labels={'supplier_name': 'Supplier', 'lead_time_days': 'Average Lead Time (days)'},
    color='lead_time_days', color_continuous_scale='Blues'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()
